In [1]:
import numpy as np
import pandas as pd
from scipy.stats import binom


In [16]:
outpath = "/scratch/PI/horence/JuliaO/single_cell/SZS_pipeline2/notebooks/output/Paper_numbers/"

## number of cells analyzed in each individual and technology

In [2]:
datanames = ["HLCA4_P2_10x_with_postprocessing_lung","HLCA4_P3_10x_with_postprocessing_lung","HLCA_smartseq_P2_with_postprocessing_shared","HLCA_smartseq_P3_with_postprocessing_shared"]
datatypes = ["10x","10x","ss2","ss2"]
individuals = ["P2","P3","P2","P3"]
ind_dict = {x : 0 for x in individuals}
tech_dict = {x : 0 for x in datatypes}
ontologies = set()
total = 0
tech_med_dict = {x : [] for x in datatypes}
for i in range(len(datanames)):
  df = pd.read_parquet("/scratch/groups/horence/JuliaO/single_cell/SZS_pipeline2/data/{}.pq".format(datanames[i]),columns=["cell","free_annotation","compartment","geneR1A_uniq"])
#   df["cell_gene"] = df["cell"] + df["geneR1A_uniq"]
  
#   df = df.drop_duplicates("cell_gene")
  df["ontology"] = df["compartment"] + df["free_annotation"]
  num_cells = df["cell"].nunique()
  print("{:,} cells {} {}, {} ontologies".format(num_cells,datatypes[i],individuals[i],df["ontology"].nunique()))
  print("median genes per cell: {:,}".format(df.groupby("cell")["geneR1A_uniq"].nunique().median()))
  tech_med_dict[datatypes[i]] += list(df.groupby("cell")["geneR1A_uniq"].nunique())
  ontologies = set.union(ontologies, set(df["ontology"].unique()))
  ind_dict[individuals[i]] += num_cells
  tech_dict[datatypes[i]] += num_cells
  total += num_cells
print()
for key, val in ind_dict.items():
  print("{}: {:,}".format(key,val))
print()
for key, val in tech_dict.items():
  print("{}: {:,}".format(key,val)) 
print()
print("total cells: {:,}".format(total))
print("total number ontologies:",len(ontologies))

print()

for key, value in tech_med_dict.items():
  print("{}: {:,}".format(key,np.median(value)))

28,793 cells 10x P2, 39 ontologies
median genes per cell: 931.0
24,676 cells 10x P3, 50 ontologies
median genes per cell: 883.0
4,217 cells ss2 P2, 31 ontologies
median genes per cell: 1,513.0
2,836 cells ss2 P3, 33 ontologies
median genes per cell: 1,819.0

P2: 33,010
P3: 27,512

10x: 53,469
ss2: 7,053

total cells: 60,522
total number ontologies: 57

10x: 912.0
ss2: 1,660.0


## Number of cells/genes with computable SpliZ

In [3]:
datanames = ["HLCA4_P2_10x_with_postprocessing_lung","HLCA4_P3_10x_with_postprocessing_lung","HLCA_smartseq_P2_with_postprocessing_shared","HLCA_smartseq_P3_with_postprocessing_shared"]
datatypes = ["10x","10x","ss2","ss2"]
individuals = ["P2","P3","P2","P3"]
tech_med_dict = {x : [] for x in datatypes}
tech_genes = {x : set() for x in datatypes}
for i in range(len(datanames)):
  df = pd.read_parquet("/scratch/PI/horence/JuliaO/single_cell/SZS_pipeline2/scripts/output/rijk_zscore/{}_sym_SVD_normdonor_S_0.1_z_0.0_b_5.pq".format(datanames[i]),columns=["cell","geneR1A_uniq"])
  print(datanames[i],df.groupby("cell")["geneR1A_uniq"].nunique().median())
  tech_med_dict[datatypes[i]] += list(df.groupby("cell")["geneR1A_uniq"].nunique())
  ser = df.groupby("geneR1A_uniq")["cell"].nunique()
  ser = ser[ser >= 10]
  print("{:,} genes".format(ser.shape[0]))
  tech_genes[datatypes[i]].update(ser.index)
for key, value in tech_med_dict.items():
  print("median genes per cell {}: {:,}".format(key,np.median(value)))
  
for key, value in tech_genes.items():
  print("num genes >= 10 {}: {}".format(key, len(value)))

HLCA4_P2_10x_with_postprocessing_lung 60.0
1,599 genes
HLCA4_P3_10x_with_postprocessing_lung 74.0
1,415 genes
HLCA_smartseq_P2_with_postprocessing_shared 782.0
10,876 genes
HLCA_smartseq_P3_with_postprocessing_shared 881.0
9,795 genes
median genes per cell 10x: 67.0
median genes per cell ss2: 833.0
num genes >= 10 10x: 1754
num genes >= 10 ss2: 11640


## called genes by the SpliZ, SpliZVD

In [8]:
inpath = "/scratch/PI/horence/JuliaO/single_cell/SZS_pipeline2/scripts/output/variance_adjusted_permutations/"
# datanames = ["HLCA4_P2_10x_with_postprocessing_lung","HLCA4_P3_10x_with_postprocessing_lung","HLCA_smartseq_P2_with_postprocessing_shared","HLCA_smartseq_P3_with_postprocessing_shared"]
# datanames = ["HLCA4_P2_10x_with_postprocessing_lung","HLCA4_P3_10x_with_postprocessing_lung"]
datanames = ["HLCA4_P2_10x_with_postprocessing_lung","HLCA4_P3_10x_with_postprocessing_lung","HLCA_smartseq_P2_with_postprocessing_shared","HLCA_smartseq_P3_with_postprocessing_shared","HLCA4_P2_10x_with_postprocessing_lung_shuffle","HLCA4_P3_10x_with_postprocessing_lung_shuffle","HLCA4_P2_10x_with_postprocessing_lung_lungimmuneMacrophage_10","HLCA4_P3_10x_with_postprocessing_lung_lungimmuneMacrophage_10"]

# suffixes = ["","_shuffle","_lungimmuneMacrophage_10"]

# datanames = ["TSP1_10x_with_postprocessing_nopanc_cellann","TSP2_10x_rerun_with_postprocessing_3prime_cellann","TS_pilot_smartseq_with_postprocessing_nopanc_cellann","TSP2_SS2_RUN1_RUN2_cellann"]
suffixes = [""]

In [9]:
z_cols = ["scZ","svd_z0"]
out_dict = {x : [] for x in z_cols}
out_dict_frac = {x : [] for x in z_cols}
out_dict["dataname"] = []
out_dict_frac["dataname"] = []
eigen_thresh = .9
gene = "PPP1R12A"
out_string = ""
for dataname in datanames:
  out_string += dataname + "\n"
  for suffix in suffixes:
    out_dict["dataname"].append(dataname + suffix)
    out_dict_frac["dataname"].append(dataname + suffix)
    print(dataname + suffix)
    df = pd.read_csv("{}{}{}_pvals_100_S_0.1_z_0.0_b_5.tsv".format(inpath,dataname,suffix),sep="\t")
    
    df["f0+f1"] = df["f0"] + df["f1"]

    for z_col in z_cols:
      print(z_col)
      if z_col == "svd_z1":
        out_dict[z_col].append(df[(df["perm_pval_adj_" + z_col] < 0.05) & (df["f0"] < eigen_thresh)]["geneR1A_uniq"].nunique())
        out_dict_frac[z_col].append(df[(df["perm_pval_adj_" + z_col] < 0.05) & (df["f0"] < eigen_thresh)]["geneR1A_uniq"].nunique()/df["geneR1A_uniq"].nunique())
        display(df[(df["perm_pval_adj_" + z_col] < 0.05) & (df["f0"] < eigen_thresh)].sort_values("max_abs_median_" + z_col).tail(10)[["geneR1A_uniq","num_onts","perm_pval_adj_" + z_col, "max_abs_median_" + z_col]])
#         df[(df["perm_pval_adj_" + z_col] < 0.05) & (df["f0"] < eigen_thresh)].to_csv("{}{}{}{}_sig.tsv".format(outpath,dataname,suffix,z_col),sep="\t",index=False)
        out_string += "{} {} in {}\n".format(z_col,gene,gene in df[(df["perm_pval_adj_" + z_col] < 0.05) & (df["f0"] < eigen_thresh)]["geneR1A_uniq"].unique())

      elif z_col == "svd_z2":
        out_dict[z_col].append(df[(df["perm_pval_adj_" + z_col] < 0.05) & (df["f0+f1"] < eigen_thresh)]["geneR1A_uniq"].nunique())
        out_dict_frac[z_col].append(df[(df["perm_pval_adj_" + z_col] < 0.05) & (df["f0+f1"] < eigen_thresh)]["geneR1A_uniq"].nunique()/df["geneR1A_uniq"].nunique())
        display(df[(df["perm_pval_adj_" + z_col] < 0.05) & (df["f0+f1"] < eigen_thresh)].sort_values("max_abs_median_" + z_col).tail(10)[["geneR1A_uniq","num_onts","perm_pval_adj_" + z_col, "max_abs_median_" + z_col]])
#         print(gene," in",gene in df[(df["perm_pval_adj_" + z_col] < 0.05) & (df["f0+f1"] < eigen_thresh)]["geneR1A_uniq"].unique())
#         df[(df["perm_pval_adj_" + z_col] < 0.05) & (df["f0+f1"] < eigen_thresh)].to_csv("{}{}{}{}_sig.tsv".format(outpath,dataname,suffix,z_col),sep="\t",index=False)

        out_string += "{} {} in {}\n".format(z_col,gene,gene in df[(df["perm_pval_adj_" + z_col] < 0.05) & (df["f0+f1"] < eigen_thresh)]["geneR1A_uniq"].unique())

      else:
        out_dict[z_col].append(df[df["perm_pval_adj_" + z_col] < 0.05]["geneR1A_uniq"].nunique())
        out_string += "{} {} in {}\n".format(z_col,gene,gene in df[(df["perm_pval_adj_" + z_col] < 0.05) ]["geneR1A_uniq"].unique())
#         df[(df["perm_pval_adj_" + z_col] < 0.05) ].to_csv("{}{}{}{}_sig.tsv".format(outpath,dataname,suffix,z_col),sep="\t",index=False)

#         print(gene," in",gene in df[df["perm_pval_adj_" + z_col] < 0.05]["geneR1A_uniq"].unique())
        out_dict_frac[z_col].append(df[df["perm_pval_adj_" + z_col] < 0.05]["geneR1A_uniq"].nunique()/df["geneR1A_uniq"].nunique())
        print("top 100 genes",list(df[df["perm_pval_adj_" + z_col] < 0.05].sort_values("max_abs_median_" + z_col)[["geneR1A_uniq","num_onts","perm_pval_adj_" + z_col, "max_abs_median_" + z_col]]["geneR1A_uniq"].tail(100)))
        display(df[df["perm_pval_adj_" + z_col] < 0.05].sort_values("max_abs_median_" + z_col).tail(10)[["geneR1A_uniq","num_onts","perm_pval_adj_" + z_col, "max_abs_median_" + z_col]])
out_df = pd.DataFrame.from_dict(out_dict)
out_frac = pd.DataFrame.from_dict(out_dict_frac)
                

HLCA4_P2_10x_with_postprocessing_lung
scZ
top 100 genes ['ZFAS1', 'RPL36A-HNRNPH2', 'S100A6', 'CD163', 'C19orf33', 'SNHG6', 'LYAR', 'RARRES1', 'PSMB2', 'HCST', 'RPL5', 'VIM', 'RPL27A', 'CMC1', 'BLVRB', 'MICOS10-NBL1', 'CCL20', 'CD63', 'JPT1', 'TXNDC17', 'S100A13', 'GMFG', 'SRSF5', 'CAST', 'YBX1', 'TMEM147', 'CYBA', 'HSP90AA1', 'HNRNPDL', 'UQCRB', 'SERF2-C15ORF63', 'GSPT1', 'ZBTB8OS', 'NDUFB3', 'CARD16', 'CP', 'NAP1L1', 'LGALS3', 'ATP5MC3', 'ITM2B', 'CALM1', 'RPAIN', 'CKLF-CMTM1', 'SRSF7', 'RPL41', 'MZT2B', 'COX4I1', 'AREG', 'ATP5MG', 'KRTCAP2', 'TMC5', 'DECR1', 'ATP5PO', 'PPFIBP1', 'HSP90B1', 'PLIN2', 'ECH1', 'CTSC', 'ARPC3', 'HNRNPA1', 'ARPC2', 'ELOC', 'RBM39', 'ARHGAP18', 'AUP1', 'SOD2', 'RPS7', 'CIRBP', 'ETHE1', 'CALD1', 'ARID4B', 'PQBP1', 'AKAP13', 'ISG20', 'PRMT1', 'SEC61G', 'ARL6IP1', 'SOD1', 'RWDD1', 'CAPG', 'LTA4H', 'MORF4L1', 'CHCHD2', 'NASP', 'LMNA', 'LRRFIP2', 'REEP3', 'RPL17', 'CD47', 'RPN2', 'NOP56', 'THYN1', 'unknown_chr7_67900000', 'SNHG8', 'SCGB1A1', 'PPP1R12A', 'MYL6',

,geneR1A_uniq,num_onts,perm_pval_adj_scZ,max_abs_median_scZ
4,NOP56,4,0.0000,0.810319
180,THYN1,2,0.0000,0.891720
190,unknown_chr7_67900000,4,0.0245,0.893863
161,SNHG8,13,0.0000,0.907007
44,SCGB1A1,16,0.0000,0.997250
12,PPP1R12A,2,0.0000,1.054181
37,MYL6,39,0.0000,1.258683
61,RPS24,39,0.0000,1.372058
24,LMO7,2,0.0000,1.874982
140,ATP5F1C,22,0.0000,2.648506


svd_z0
top 100 genes ['RPS25', 'MORF4L1', 'RAB2A', 'ATP5PD', 'RPL24', 'SLTM', 'HLA-DQA1', 'CALD1', 'RPL23', 'ITM2B', 'VIM', 'RPL30', 'PTMA', 'CCDC91', 'NDUFAF3', 'SAT1', 'HPGD', 'MYL6', 'UXT', 'CIAO2B', 'GAS5', 'IFI30', 'S100A13', 'PSME2', 'COX7C', 'RPN2', 'PRDX5', 'PLIN2', 'FABP4', 'TPM2', 'LGALS1', 'NDUFB8', 'NAP1L1', 'ARHGAP18', 'RPS13', 'RPS8', 'RPL28', 'NASP', 'CAST', 'RPS27A', 'ATP5F1C', 'RBM39', 'RPS15A', 'RACK1', 'BPIFB1', 'POLR2I', 'RPSA', 'PFDN5', 'LRRFIP1', 'RPS12', 'AREG', 'PSMB7', 'TCIRG1', 'RPL27A', 'DEK', 'MGST3', 'RPL35A', 'HSP90AA1', 'NOP56', 'DNAJC15', 'RNF130', 'LGALS3', 'SELENOH', 'ATP5MC3', 'IGFBP7', 'RPS19', 'CALM1', 'RPS6', 'RPL10A', 'S100A6', 'CYBA', 'TPT1', 'DECR1', 'TBCA', 'S100A9', 'CLU', 'RPLP2', 'TYROBP', 'RPS3', 'BLOC1S1-RDH5', 'RPS7', 'CD63', 'TXN', 'KRTCAP2', 'POMP', 'SCGB1A1', 'RPS4X', 'MARCO', 'ANXA1', 'LCN2', 'MT1E', 'RPL12', 'RPS2', 'UBA52', 'RPS14', 'SCGB3A2', 'RPL13A', 'RPL17', 'FTL', 'SCGB3A1']


,geneR1A_uniq,num_onts,perm_pval_adj_svd_z0,max_abs_median_svd_z0
571,MT1E,18,0.0,5.671791
731,RPL12,39,0.0,5.708614
754,RPS2,39,0.0,5.957620
109,UBA52,39,0.0,6.878262
59,RPS14,39,0.0,7.027029
779,SCGB3A2,5,0.0,7.084992
80,RPL13A,39,0.0,7.735757
82,RPL17,39,0.0,8.841513
166,FTL,39,0.0,9.722942
32,SCGB3A1,14,0.0,46.593897


HLCA4_P3_10x_with_postprocessing_lung
scZ
top 100 genes ['SPARCL1', 'RPS27L', 'CIRBP', 'KMT2E', 'RPL3', 'ISG20', 'PLEKHA5', 'NDUFA4', 'SRSF5', 'NDUFB8', 'CLU', 'AREG', 'SEC61G', 'CD74', 'RPS3', 'LY96', 'CTNNAL1', 'HNRNPDL', 'TPT1', 'ATP5MC3', 'GMFG', 'SNRPB', 'ATP5PD', 'CIAO2B', 'MGST3', 'S100A13', 'RPS7', 'HNRNPA1', 'FAM183A', 'TYROBP', 'DPP7', 'KYNU', 'MAP4K1', 'EIF3G', 'TMEM258', 'ERGIC3', 'CAST', 'LTA4H', 'PLTP', 'BLVRB', 'ARL6IP1', 'HCST', 'ECSCR', 'GSTP1', 'ERLEC1', 'PSMB1', 'DECR1', 'TMEM147', 'TREM1', 'EMP3', 'CHCHD2', 'MNDA', 'SOD1', 'MICOS10-NBL1', 'COX5B', 'CAPG', 'RPL38', 'WFDC21P', 'LYAR', 'FTL', 'EMP2', 'NDUFAF3', 'JPT1', 'LGALS3', 'CMC1', 'ARPC2', 'MSRA', 'CD63', 'IFT57', 'COX4I1', 'KRTCAP2', 'ITM2B', 'AUP1', 'ATP5PO', 'NDUFS2', 'SCGB1A1', 'DBI', 'SRSF7', 'CRNDE', 'ATP5MG', 'UQCRB', 'LMNA', 'DAD1', 'THYN1', 'METTL26', 'RPL4', 'RPN2', 'ARPC3', 'RWDD1', 'FYB1', 'NUPR1', 'CD55', 'PRMT1', 'RPS24', 'CD47', 'MYL6', 'PPP1R12A', 'CIR1', 'ANXA1', 'ATP5F1C']


...

,geneR1A_uniq,num_onts,perm_pval_adj_svd_z0,max_abs_median_svd_z0
484,RAB13,10,0.0,1.732683
1,NUPR1,10,0.0,3.016532
316,LY75-CD302,10,0.0,3.058669


HLCA4_P3_10x_with_postprocessing_lung_lungimmuneMacrophage_10
scZ
top 100 genes []


,geneR1A_uniq,num_onts,perm_pval_adj_scZ,max_abs_median_scZ


svd_z0
top 100 genes ['LOC101928215', 'UBE2D2', 'CD163', 'HPGDS']


,geneR1A_uniq,num_onts,perm_pval_adj_svd_z0,max_abs_median_svd_z0
31,LOC101928215,7,0.0,1.488937
11,UBE2D2,10,0.0,2.143626
107,CD163,10,0.0,2.690212
242,HPGDS,2,0.0,2.760181


In [10]:
num_genes = out_df[["dataname"] + z_cols]
num_genes

,dataname,scZ,svd_z0
0,HLCA4_P2_10x_with_postprocessing_lung,207,135
1,HLCA4_P3_10x_with_postprocessing_lung,219,142
2,HLCA_smartseq_P2_with_postprocessing_shared,606,641
3,HLCA_smartseq_P3_with_postprocessing_shared,336,361
4,HLCA4_P2_10x_with_postprocessing_lung_shuffle,1,6
5,HLCA4_P3_10x_with_postprocessing_lung_shuffle,3,7
6,HLCA4_P2_10x_with_postprocessing_lung_lungimmu...,0,3
7,HLCA4_P3_10x_with_postprocessing_lung_lungimmu...,0,4


In [11]:
frac_genes = out_frac[["dataname"] + z_cols]
frac_genes

,dataname,scZ,svd_z0
0,HLCA4_P2_10x_with_postprocessing_lung,0.228225,0.148842
1,HLCA4_P3_10x_with_postprocessing_lung,0.254060,0.164733
2,HLCA_smartseq_P2_with_postprocessing_shared,0.088545,0.093659
3,HLCA_smartseq_P3_with_postprocessing_shared,0.059989,0.064453
4,HLCA4_P2_10x_with_postprocessing_lung_shuffle,0.000900,0.005401
5,HLCA4_P3_10x_with_postprocessing_lung_shuffle,0.003344,0.007804
6,HLCA4_P2_10x_with_postprocessing_lung_lungimmu...,0.000000,0.004093
7,HLCA4_P3_10x_with_postprocessing_lung_lungimmu...,0.000000,0.006250


In [13]:
name_dict = {x : y for x, y in zip(datanames, ["Ind 1 10x", "Ind 2 10x", "Ind 1 SS2 shared", "Ind 2 SS2 shared","Ind 1 shuffle", "Ind 2 shuffle", 
                                               "Ind 1 macrophage", "Ind 2 macrophage"])}

In [24]:
out = num_genes.merge(frac_genes,on="dataname")
out["dataset"] = out["dataname"].map(name_dict)
out = out.rename(columns={"scZ_x" : "# sig by SpliZ", "svd_z0_x" : "# sig by SpliZVD", "scZ_y" : "frac sig by SpliZ", "svd_z0_y" : "frac sig by SpliZVD"})
out = out[["dataset","# sig by SpliZ","frac sig by SpliZ",  "# sig by SpliZVD","frac sig by SpliZVD"]].round(3)
out.to_csv("{}Supp_Table2.csv".format(outpath),index=False)
out

,dataset,# sig by SpliZ,frac sig by SpliZ,# sig by SpliZVD,frac sig by SpliZVD
0,Ind 1 10x,207,0.228,135,0.149
1,Ind 2 10x,219,0.254,142,0.165
2,Ind 1 SS2 shared,606,0.089,641,0.094
3,Ind 2 SS2 shared,336,0.060,361,0.064
4,Ind 1 shuffle,1,0.001,6,0.005
5,Ind 2 shuffle,3,0.003,7,0.008
6,Ind 1 macrophage,0,0.000,3,0.004
7,Ind 2 macrophage,0,0.000,4,0.006


In [7]:
datanames

['HLCA4_P2_10x_with_postprocessing_lung',
 'HLCA4_P3_10x_with_postprocessing_lung',
 'HLCA4_P2_10x_with_postprocessing_lung_shuffle',
 'HLCA4_P3_10x_with_postprocessing_lung_shuffle',
 'HLCA4_P2_10x_with_postprocessing_lung_lungimmuneMacrophage_10',
 'HLCA4_P3_10x_with_postprocessing_lung_lungimmuneMacrophage_10',
 'HLCA_smartseq_P2_with_postprocessing_shared',
 'HLCA_smartseq_P3_with_postprocessing_shared']

## Intersection of genes called as significant

In [16]:
genes = ["CD47","TPM2","PPP1R12A"]
datanames = []
# genes = ["CD47"]
suffix = ""
datanames = ["HLCA4_P2_10x_with_postprocessing_lung","HLCA4_P3_10x_with_postprocessing_lung","HLCA_smartseq_P2_with_postprocessing_shared","HLCA_smartseq_P3_with_postprocessing_shared"]

z_cols = ["scZ","svd_z0"]
for dataname in datanames:
  print(dataname)
#   out_string += dataname + "\n"

#   out_dict["dataname"].append(dataname + suffix)
#   out_dict_frac["dataname"].append(dataname + suffix)
#   print(dataname + suffix)
  df = pd.read_csv("{}{}{}_pvals_100_S_0.1_z_0.0_b_5.tsv".format(inpath,dataname,suffix),sep="\t")
  print("genes shared by SpliZ and SpliZVD: {}".format(df[(df["perm_pval_adj_scZ"] < 0.05) & (df["perm_pval_adj_svd_z0"] < 0.05)]["geneR1A_uniq"].nunique()))
  for gene in genes:
    print("gene: {}".format(gene))
    for z_col in z_cols:
      print("{}: {}".format(z_col,df[df["geneR1A_uniq"] == gene]["perm_pval_adj_"+ z_col].iloc[0]))
      df = df.sort_values("max_abs_median_" + z_col,ascending=False)
      df.reset_index(inplace=True,drop=True)
      print("rank: {}".format(df[df["geneR1A_uniq"] == gene].index[0]))
    print()
  #   break

HLCA4_P2_10x_with_postprocessing_lung
genes shared by SpliZ and SpliZVD: 88
gene: CD47
scZ: 0.0
rank: 12
svd_z0: 0.0
rank: 252

gene: TPM2
scZ: 0.0
rank: 266
svd_z0: 0.0
rank: 121

gene: PPP1R12A
scZ: 0.0
rank: 4
svd_z0: 0.0
rank: 387

HLCA4_P3_10x_with_postprocessing_lung
genes shared by SpliZ and SpliZVD: 91
gene: CD47
scZ: 0.0
rank: 5
svd_z0: 0.0
rank: 304

gene: TPM2
scZ: 0.0
rank: 204
svd_z0: 0.0
rank: 82

gene: PPP1R12A
scZ: 0.0
rank: 3
svd_z0: 0.0
rank: 164

HLCA_smartseq_P2_with_postprocessing_shared
genes shared by SpliZ and SpliZVD: 338
gene: CD47
scZ: nan
rank: 5256
svd_z0: nan
rank: 2531

gene: TPM2
scZ: nan
rank: 5065
svd_z0: 0.21977292576419236
rank: 245

gene: PPP1R12A
scZ: nan
rank: 2909
svd_z0: nan
rank: 3250

HLCA_smartseq_P3_with_postprocessing_shared
genes shared by SpliZ and SpliZVD: 192
gene: CD47
scZ: nan
rank: 3904
svd_z0: nan
rank: 670

gene: TPM2
scZ: nan
rank: 1687
svd_z0: nan
rank: 2376

gene: PPP1R12A
scZ: nan
rank: 1504
svd_z0: 0.0
rank: 2006



In [17]:
for individual in ["P2","P3"]:
  print(individual)
  tenx_df =  pd.read_csv("{}HLCA4_{}_10x_with_postprocessing_lung_pvals_100_S_0.1_z_0.0_b_5.tsv".format(inpath,individual),sep="\t")
  ss2_df =  pd.read_csv("{}HLCA_smartseq_{}_with_postprocessing_shared_pvals_100_S_0.1_z_0.0_b_5.tsv".format(inpath,individual),sep="\t")
  tenx_sig = set(tenx_df[(tenx_df["perm_pval_adj_scZ"] < 0.05) | (tenx_df["perm_pval_adj_svd_z0"] < 0.05)]["geneR1A_uniq"].unique())
  ss2_sig = set(ss2_df[(ss2_df["perm_pval_adj_scZ"] < 0.05) | (ss2_df["perm_pval_adj_svd_z0"] < 0.05)]["geneR1A_uniq"].unique())
  print("number significant 10x: {:,}".format(len(tenx_sig)))
  print("fraction significant 10x: {}".format(len(tenx_sig)/tenx_df.shape[0]))
  print("number signfiicant ss2: {:,}".format(len(ss2_sig)))
  print("fraction significant ss2: {}".format(len(ss2_sig)/ss2_df.shape[0]))
  
  print("{} intersection: {}".format(individual,len(tenx_sig.intersection(ss2_sig))))
  print("pvalue: {}".format(1 - binom.cdf(len(tenx_sig.intersection(ss2_sig)),tenx_df.shape[0],(len(tenx_sig)/tenx_df.shape[0])*(len(ss2_sig)/ss2_df.shape[0]))))
# 1 - binom.cdf(get_num_genes(df,True,True,True), df.shape[0], p_sig[0]*p_sig[1]*p_sig[2])

P2
number significant 10x: 254
fraction significant 10x: 0.28004410143329656
number signfiicant ss2: 909
fraction significant ss2: 0.13281706604324955
P2 intersection: 73
pvalue: 5.713001183238475e-10
P3
number significant 10x: 270
fraction significant 10x: 0.31322505800464034
number signfiicant ss2: 505
fraction significant ss2: 0.09016247098732369
P3 intersection: 46
pvalue: 2.1905508912567484e-05


## Concordance 10x vs 10x

In [10]:
df_P2 =  pd.read_csv("{}HLCA4_P2_10x_with_postprocessing_lung_pvals_100_S_0.1_z_0.0_b_5.tsv".format(inpath),sep="\t")
df_P3 =  pd.read_csv("{}HLCA4_P3_10x_with_postprocessing_lung_pvals_100_S_0.1_z_0.0_b_5.tsv".format(inpath),sep="\t")


In [32]:
P2_set = set(df_P2[df_P2["perm_pval_adj_scZ"] < 0.05]["geneR1A_uniq"].unique())
P3_set = set(df_P3[df_P3["perm_pval_adj_scZ"] < 0.05]["geneR1A_uniq"].unique())
print("spliz intersection: {}".format(len(P2_set.intersection(P3_set))))

spliz intersection: 136


In [33]:
P2_set2 = set(df_P2[df_P2["perm_pval_adj_svd_z0"] < 0.05]["geneR1A_uniq"].unique())
P3_set2 = set(df_P3[df_P3["perm_pval_adj_svd_z0"] < 0.05]["geneR1A_uniq"].unique())
print("splizVD intersection: {}".format(len(P2_set2.intersection(P3_set2))))

splizVD intersection: 81


In [34]:
len(P2_set2 - P2_set)

47

In [35]:
len(P3_set2 - P3_set)

51

In [13]:
P2_set = set(df_P2[(df_P2["perm_pval_adj_scZ"] < 0.05) | (df_P2["perm_pval_adj_svd_z0"] < 0.05)]["geneR1A_uniq"].unique())
P3_set = set(df_P3[(df_P3["perm_pval_adj_scZ"] < 0.05) | (df_P3["perm_pval_adj_svd_z0"] < 0.05) ]["geneR1A_uniq"].unique())
print("either intersection: {}".format(len(P2_set.intersection(P3_set))))

either intersection: 175


In [14]:
print("pvalue: {}".format(1 - binom.cdf(len(P2_set.intersection(P3_set)),df_P2.shape[0],(len(P2_set)/df_P2.shape[0])*(len(P3_set)/df_P3.shape[0]))))


pvalue: 1.1102230246251565e-16
